# https://www.youtube.com/watch?v=4uHsEgmiI-Y&list=PLxhYs0VkONL9h5zHZXNntPqLC1JVg9pxX&index=145

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, Callback



In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
X_train, X_test = X_train / 255.0, X_test / 255.0

In [5]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_checkpoint = ModelCheckpoint(
    'model.keras',
    verbose=2,
    save_best_only=True,
)

In [18]:
early_stopping_cb = EarlyStopping(
    patience=2,
    # verbose=2,
)

In [27]:
import math
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, learning_rate):
    print(f'Epoch: {epoch}, Learning Rate: {learning_rate}')
    if epoch < 5:
        return learning_rate
    else:
        return learning_rate * math.exp(-0.1)

lr_scheduler_cb = LearningRateScheduler(scheduler)

In [38]:
class CustomLogger(Callback):
    def __init__(self, logfile):
        super().__init__()
        self.logfile = logfile
        
    def on_epoch_end(self, epoch, logs):
        with open(self.logfile, 'a') as f:
            f.write(f'Epoch: {epoch}, Loss: {logs["loss"]}, Accuracy: {logs["accuracy"]}, Val Loss: {logs["val_loss"]}, Val Accuracy: {logs["val_accuracy"]}\n')

In [39]:
custom_logger_cb = CustomLogger('log.txt')

In [40]:
tensorboard_cb = TensorBoard(
    'logs',
    update_freq=1,
)

In [41]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[model_checkpoint, early_stopping_cb, lr_scheduler_cb])

Epoch: 0, Learning Rate: 0.009999999776482582
Epoch 1/50
1856/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.9711 - loss: 0.1301
Epoch 1: val_loss did not improve from 0.19729
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9711 - loss: 0.1302 - val_accuracy: 0.9657 - val_loss: 0.4288 - learning_rate: 0.0100
Epoch: 1, Learning Rate: 0.009999999776482582
Epoch 2/50
1859/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.9725 - loss: 0.1265
Epoch 2: val_loss did not improve from 0.19729
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9725 - loss: 0.1265 - val_accuracy: 0.9641 - val_loss: 0.4253 - learning_rate: 0.0100


# tensorboard --logdir=logs
